In [1]:
import matplotlib.pyplot as plt
import cv2
import re
import os
import shutil

In [2]:
import numpy as np
import pickle as pkl
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from PIL import Image

In [ ]:
# This is required for me to have autocomplete
%config Completer.use_jedi = False
%matplotlib inline

!jupyter nbextension enable --py widgetsnbextension

In [3]:
# set data path for images
image_path = "data/testset/images/"

# set data path for annotations
annotation_path = "data/testset/annotations/"

# set path for predictions
predict_path = 'data/testset/predictions/'

# array path
array_path = "data/testset/final_arrays/"

# evaluation path
eval_path = "data/testset/evaluation/"

# dataset sizes (for looping)
sizes = ["small","medium","large","small_transform","medium_transform","large_transform"]

# models
models = ["VGG32", 
          "VGG8", 
          "Vanilla8", 
          "Vanilla32",
          "Restnet8",
          "restnet32",
         "VanillaUnet",
         "Resnet50Unet"]

## Convert annotations to dict of 2D arrays

In [6]:
# convert annotations to 2D arrays
array_dict = {}
annotations = os.listdir(annotation_path)

In [7]:
for ann in annotations:
    image = cv2.imread(annotation_path+ann)
    
    # sum to a 2D array
    segment_array = np.sum(image, axis=2)
    
    # set foreground to 100
    segment_array = np.where(segment_array == 298, 1, segment_array)
    
    # add to dict
    array_dict[ann] = segment_array

In [8]:
# save dict
with open(array_path+'annotations.pickle', 'wb') as handle:
    pkl.dump(array_dict, handle, protocol=pkl.HIGHEST_PROTOCOL)

## Convert predictions to dict of 2D arrays

In [5]:
for model in models:
    for size in sizes:
        model_dir = predict_path+model+"/"+size+"/"
        
        images = os.listdir(model_dir)
        array_dict = {}
        
        for im in images:
            image = cv2.imread(model_dir+im)
    
            # sum to a 2D array
            im_array = np.sum(image, axis=2)

            # set foreground to 1, background to 0
            im_array = np.where(im_array == 587, 1, im_array)
            im_array = np.where(im_array == 432, 0, im_array)
            
            # add to dict
            array_dict[im] = im_array
        
        # save dict
        with open(array_path+model+"_"+size+'.pickle', 'wb') as handle:
            pkl.dump(array_dict, handle, protocol=pkl.HIGHEST_PROTOCOL)